In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
import os
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/tmlc_images.zip"

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create a new generator
datagen_train = ImageDataGenerator(rotation_range=45, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest", rescale=1./255)
datagen = ImageDataGenerator(rescale = 1./255)
# load train data
train_dir =  "train"
# load validation data
val_dir = "val"
# load train data
test_dir = "test"
train = datagen_train.flow_from_directory(train_dir, target_size = (224,224),batch_size = 32, class_mode = 'categorical')
val = datagen.flow_from_directory(val_dir, target_size = (224,224),batch_size = 32, class_mode = 'categorical')
#test = datagen.flow_from_directory(train_dir, target_size = (100,100),batch_size = 100, class_mode = categorical)

Found 12753 images belonging to 104 classes.
Found 3712 images belonging to 104 classes.


## Xception model

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras import regularizers
pre_trained_model = Xception(input_shape=(224,224,3), include_top=False, weights="imagenet")
#pre_trained_model.trainable = False

for layer in pre_trained_model.layers[:21]:
    layer.trainable = False
model = Sequential([
    pre_trained_model,
    MaxPool2D((2,2) , strides = 2),
    Flatten(),
    #Dropout(0.2),
    Dense(100, activation = 'relu'),
    Dropout(0.25),
    Dense(64, activation = 'relu'),
    Dropout(0.3),
    Dense(128, activation = 'relu'),#kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.25),
    Dense(104 , activation='softmax')])
model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
model.fit(train , epochs = 30 , validation_data = val)

In [ ]:
model.save("flowers_79.h5")

In [ ]:
loss, acc = model.evaluate(val, verbose=0)

In [ ]:
acc #accuracy of Xception model

0.798491358757019

## VGG-19 model

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras import regularizers
pre_trained_model = VGG19(input_shape=(224,224,3), include_top=False, weights="imagenet")
#pre_trained_model.trainable = False

for layer in pre_trained_model.layers[:19]:
    layer.trainable = False
model = Sequential([
    pre_trained_model,
    MaxPool2D((2,2) , strides = 2),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dropout(0.25),
    Dense(128, activation = 'relu'),
    Dropout(0.2),
    Dense(104 , activation='softmax')])
model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
 model.fit(train , epochs = 50 , validation_data = val)

In [ ]:
loss, acc = model.evaluate(val, verbose=0)

In [ ]:
acc #accuracy of VGG-19 model

0.6896551847457886